In [2]:
!pip show opendartreader

Name: OpenDartReader
Version: 0.2.3
Summary: OpenDartReader (FSS DART Open API reader)
Home-page: https://github.com/financedata/opendartreader
Author: FinanceData.KR
Author-email: plusjune@financedata.kr
License: MIT License
Location: c:\users\admin\miniforge3\envs\fintech\lib\site-packages
Requires: beautifulsoup4, lxml, pandas, requests, requests-file
Required-by: 


In [8]:
!pip uninstall opendartreader -y
!pip install opendartreader

Found existing installation: OpenDartReader 0.2.3
Uninstalling OpenDartReader-0.2.3:
  Successfully uninstalled OpenDartReader-0.2.3
  Using cached OpenDartReader-0.2.3-py3-none-any.whl.metadata (913 bytes)
Using cached OpenDartReader-0.2.3-py3-none-any.whl (27 kB)


In [10]:
import OpenDartReader

dart = OpenDartReader("57843872c61b2852422b4870c0a618d09ae430f7")
print("✅ DART 연결 성공")

✅ DART 연결 성공


In [17]:
import OpenDartReader
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

# 🔑 API KEY
API_KEY = OpenDartReader("57843872c61b2852422b4870c0a618d09ae430f7")
BASE_URL = "https://opendart.fss.or.kr/api/list.json"

# 📅 최근 3년
end_date = datetime.today()
start_date = end_date - timedelta(days=365 * 3)

params = {
    "crtfc_key": API_KEY,
    "bgn_de": start_date.strftime("%Y%m%d"),
    "end_de": end_date.strftime("%Y%m%d"),
    "page_no": 1,
    "page_count": 100,
}

all_results = []

# ======================================================
# 2. 페이지네이션 조회
# ======================================================
while True:
    res = requests.get(BASE_URL, params=params).json()

    if res.get("status") != "000":
        print("API 오류:", res)
        break

    df = pd.DataFrame(res.get("list", []))

    # 🔥 증권발행실적보고서만 필터
    df = df[df["report_nm"].str.contains("증권발행실적보고서", na=False)]
    all_results.append(df)

    if params["page_no"] >= int(res.get("total_page", 1)):
        break

    params["page_no"] += 1
    time.sleep(0.3)  # 서버 보호 + 안정성

# ======================================================
# 3. 결과 통합
# ======================================================
result_df = pd.concat(all_results, ignore_index=True)

# ======================================================
# 4. 컬럼 정리 + 링크 생성
# ======================================================
result_df["rcept_dt"] = pd.to_datetime(result_df["rcept_dt"])
result_df["보고서_URL"] = (
    "https://dart.fss.or.kr/dsaf001/main.do?rcpNo="
    + result_df["rcept_no"]
)

result_df = result_df[
    [
        "corp_name",
        "corp_code",
        "report_nm",
        "rcept_dt",
        "rcept_no",
        "보고서_URL",
    ]
]

print(f"✅ 최근 3년 증권발행실적보고서 {len(result_df)}건")

# ======================================================
# 5. 엑셀 저장
# ======================================================
result_df.to_excel(
    "최근3년_전체_증권발행실적보고서.xlsx",
    index=False
)

print("📁 엑셀 저장 완료")
result_df.head()

API 오류: {'status': '100', 'message': '잘못된 인증키입니다.발급받은 인증키(40자리)'}


ValueError: No objects to concatenate